# Amazon Fine-Food Reviews Dataset

## Summary of the dataset:

* The data consist of 568454 rows and 10 columns.

* The data consist of following information.
    * The unique id of each product(ProductId)
    * Unique Id of each user(UserId)
    * a rating from 1 to 5(Score)
    * time at which review was posted(Time)
    * summary of the review given(Summary)
    * review given by the customer(Text)
* Columns that were removed.
    * profile name - (the nick name of the user)
    * Helpfulness numerator - (was the review helpful to others)
    * Helpfulness denominator - (helpfull+ not helpful)
    
* Column Data-types:
    * ProductId - object type
    * UserId - object type
    * Score - int(1-5)
    * Time - time stamp
    * summary - text data
    * text - text data
* There are 27 null values in summary column and else everything is non-null.

In [0]:
#importing necessary libraries
import re
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
#reading data
df=pd.read_csv('/content/drive/My Drive/Datasets/Amazon Fine Food /amazon-fine-food-reviews/Reviews.csv')

In [0]:
df=df[df['Score']!=3]

In [0]:
df.tail()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
568449,568450,B001EO7N10,A28KG5XORO54AY,Lettie D. Carter,0,0,5,1299628800,Will not do without,Great for sesame chicken..this is a good if no...
568450,568451,B003S1WTCU,A3I8AFVPEE8KI5,R. Sawyer,0,0,2,1331251200,disappointed,I'm disappointed with the flavor. The chocolat...
568451,568452,B004I613EE,A121AA1GQV751Z,"pksd ""pk_007""",2,2,5,1329782400,Perfect for our maltipoo,"These stars are small, so you can give 10-15 o..."
568452,568453,B004I613EE,A3IBEVCTXKNOH,"Kathy A. Welch ""katwel""",1,1,5,1331596800,Favorite Training and reward treat,These are the BEST treats for training and rew...
568453,568454,B001LR2CU2,A3LGQPJCZVL9UC,srfell17,0,0,5,1338422400,Great Honey,"I am very satisfied ,product is as advertised,..."


## 1. Data Deduplication 
* Some of the reviews are repated so they need to removed.
* Reviews done at the same time for same product, by same user and the same review with same summary.

In [0]:
# checking for null values
import numpy as np
np.sum(df.isna())

Id                         0
ProductId                  0
UserId                     0
ProfileName               16
HelpfulnessNumerator       0
HelpfulnessDenominator     0
Score                      0
Time                       0
Summary                   25
Text                       0
dtype: int64

In [0]:
# droping the Profile name column as a user can be identified by its user id only 
df.drop('ProfileName',axis=1,inplace=True)

In [0]:
df.head()

,Id,ProductId,UserId,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [0]:
df[df['UserId']=='A3HDKO7OW0QNK4']

,Id,ProductId,UserId,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
10,11,B0001PB9FE,A3HDKO7OW0QNK4,1,1,5,1107820800,The Best Hot Sauce in the World,I don't know if it's the cactus or the tequila...
29,30,B0001PB9FY,A3HDKO7OW0QNK4,1,1,5,1107820800,The Best Hot Sauce in the World,I don't know if it's the cactus or the tequila...
195371,195372,B0001PB9EK,A3HDKO7OW0QNK4,0,0,5,1107907200,Authentic Mexican Restaurant Taste,Just as with the Original Tequila Picante Gour...
506516,506517,B0001PB9EA,A3HDKO7OW0QNK4,0,0,5,1107907200,Authentic Mexican Restaurant Taste,Just as with the Original Tequila Picante Gour...


### Observation:
* we see that some reviews are written at the same time by same person and review wriiten is also on further analysis we found that the product was same with different flavours.
* Example:
    * AR5J8UI46CURR	- summary(LOACKER QUADRATINI VANILLA WAFERS) 
    this user gives 5 reviews that were all the same but only the product Id was different.
    * A3HDKO7OW0QNK4 - (The Best Hot Sauce in the World)
    the user gives two reviews that are same but only the product Id is different


In [0]:
# so we removed duplicate columns:
n=len(df)
df.drop_duplicates({'UserId','Score','Time','Text'},inplace=True)

In [0]:
# size of new df
print(f'Size of new Dataset is {df.shape}')
print('so cleaned data is {:.2f}% of the orignal data'.format(len(df)/n*100))

Size of new Dataset is (364183, 9)
so cleaned data is 69.26% of the orignal data


In [0]:
#Number of positive and negative points in the dataset
df['Score'].value_counts()

5    250956
4     56113
1     36305
2     20809
Name: Score, dtype: int64

In [0]:
#categorizing score into two categories 0- less than 3 and 1 - grater than 3.
arr=[]
for i in df['Score']:
    if i>3:
        arr.append(1)
    else:
        arr.append(0)
df['Score']=arr

In [0]:
df['Score'].value_counts()

1    307069
0     57114
Name: Score, dtype: int64

## 2. Text Data cleaning
* You would have heard a lot of time in ML 'Garbarge in, Garbage Out'. If you give garbage so you will get garbage only so you need to first process the data.
* So as we cleaned the data we need to clean the text data also.(i.e review and summary column).
* The objectives of text cleaning are:
    * Removing html tags.
    * Removing punctutations.
    * Remove Special characters.
    * Keping only alphabets.
    * Converting the words to lower case.
    * Expand the contracted words e.g - haven't --> have not
    * Removing stop words.
    * Performing stemming
    
    Note: It has been researched that all the adjectives are of more than 3 letters, so remove all words with length <=3.

In [0]:
import re
def rem_HtmlTag(s):
    return re.sub(r'<.*?>',r'',s)

def rem_punctuation(s):
    return re.sub(r'[.?!,;:\[\]{}()`\'\-"]',r' ',s)

def rem_specialChar(s):
    return re.sub(r'[@#$%^&*+-\/\\]',r'',s)

def rem_alphanum(s):  
    return re.sub(r'[^A-Za-z]',r'',s)

def de_cont(s):
    try:
        contractions = { 
"ain't": "am not ","aren't": "are not","can't": "cannot","can't've": "cannot have","'cause": "because","could've": "could have","couldn't": "could not","couldn't've": "could not have","didn't": "did not",
"doesn't": "does not","don't": "do not","hadn't": "had not","hadn't've": "had not have","hasn't": "has not","haven't": "have not","he'd": "he had / he would",
"he'd've": "he would have","he'll": "he shall / he will","he'll've": "he shall have / he will have","he's": "he has / he is","how'd": "how did","how'd'y": "how do you","how'll": "how will","how's": "how has / how is / how does","I'd": "I had / I would","I'd've": "I would have","I'll": "I shall / I will","I'll've": "I shall have / I will have","I'm": "I am","I've": "I have","isn't": "is not","it'd": "it had / it would","it'd've": "it would have","it'll": "it shall / it will","it'll've": "it shall have / it will have","it's": "it has / it is","let's": "let us","ma'am": "madam","mayn't": "may not","might've": "might have","mightn't": "might not","mightn't've": "might not have","must've": "must have","mustn't": "must not","mustn't've": "must not have","needn't": "need not","needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not","oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not","shan't've": "shall not have","she'd": "she had / she would","she'd've": "she would have","she'll": "she shall / she will","she'll've": "she shall have / she will have","she's": "she has / she is","should've": "should have","shouldn't": "should not","shouldn't've": "should not have","so've": "so have","so's": "so as / so is","that'd": "that would / that had","that'd've": "that would have","that's": "that has / that is",
"there'd": "there had / there would","there'd've": "there would have","there's": "there has / there is","they'd": "they had / they would","they'd've": "they would have","they'll": "they shall / they will","they'll've": "they shall have / they will have","they're": "they are","they've": "they have","to've": "to have","wasn't": "was not","we'd": "we had / we would","we'd've": "we would have","we'll": "we will","we'll've": "we will have","we're": "we are","we've": "we have","weren't": "were not","what'll": "what shall / what will","what'll've": "what shall have / what will have","what're": "what are","what's": "what has / what is","what've": "what have","when's": "when has / when is","when've": "when have","where'd": "where did","where's": "where has / where is","where've": "where have","who'll": "who shall / who will","who'll've": "who shall have / who will have","who's": "who has / who is","who've": "who have","why's": "why has / why is","why've": "why have","will've": "will have","won't": "will not","won't've": "will not have","would've": "would have","wouldn't": "would not","wouldn't've": "would not have","y'all": "you all","y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you had / you would","you'd've": "you would have","you'll": "you shall / you will","you'll've": "you shall have / you will have","you're": "you are","you've": "you have"
}
        return contractions[s] 
    except:
        return s
nltk.download('stopwords')
lemma=nltk.stem.WordNetLemmatizer()
stop_words=stopwords.words('english')
stop_words.remove('not')
stop_words.remove('nor')
stop_words.remove('no')
def rem_stop_words(s):
    if s not in stop_words and len(s)>2:
        return True
    return False
    


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
def call_all(s):
    return rem_specialChar(rem_punctuation(rem_HtmlTag(s)))

In [0]:
cleaned_data=[]
nltk.download('wordnet')
# cleaning the text-data(review text)
for i in tqdm(df['Text']):
    temp=''
    for j in call_all(i).split():
        j=lemma.lemmatize(rem_alphanum(de_cont(j.lower())))
        if rem_stop_words(j):
            temp+=j+' '
    cleaned_data.append(temp)
df['Clean_Text']=cleaned_data


  0%|          | 0/364183 [00:00<?, ?it/s]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


100%|██████████| 364183/364183 [05:25<00:00, 1120.01it/s]


In [0]:
#cleaning the summary column
cleaned_data=[]
df['Summary'].fillna('',inplace=True)
df['Summary'].isna().value_counts()
for i in tqdm(df['Summary']):
    temp=''
    for j in call_all(i).split():
        j=lemma.lemmatize(rem_alphanum(de_cont(j.lower())))
        if rem_stop_words(j):
            temp+=j+' '
    cleaned_data.append(temp)
df['clean_summary']=cleaned_data

100%|██████████| 364183/364183 [00:18<00:00, 19229.39it/s]


In [0]:
# Length Comparison of cleaned data words and un-cleaned data words

In [0]:
#Summary
old_len=[]
new_len=[]
for i in tqdm(range(len(df))):
    old_len.append(len(df.iloc[i]['Summary'].split()))
    new_len.append(len(df.iloc[i]['clean_summary'].split()))

print('Mean and median of previous lengths')
print(np.mean(old_len))
print(np.median(old_len))

print('\nMean and Median of clean data length')
print(np.mean(new_len))
print(np.median(new_len))


fig, axs = plt.subplots(ncols=2)
plt.figure(1)
plt.subplot(121)
ax=sns.distplot(old_len,kde=True,hist=False,label="Old length")
plt.subplot(122)
ax=sns.distplot(new_len,hist=False,label="New length")


 39%|███▉      | 143119/364183 [00:53<01:20, 2738.47it/s]

In [0]:
#Text 
old_len=[]
new_len=[]
for i in tqdm(range(len(df))):
    old_len.append(len(df.iloc[i]['Text'].split()))
    new_len.append(len(df.iloc[i]['Clean_Text'].split()))


print('Mean and median of previous lengths')
print(np.mean(old_len))
print(np.median(old_len))

print('\nMean and Median of clean data length')
print(np.mean(new_len))
print(np.median(new_len))

fig, axs = plt.subplots(ncols=2)
plt.figure(1)
plt.subplot(121)
ax=sns.distplot(old_len,kde=True,hist=False,label="Old length")
plt.subplot(122)
ax=sns.distplot(new_len,hist=False,label="New length")
plt.show()


In [0]:
# now we have cleaned the text so we need to remove the rows that were having garbage
df.head()
df.drop(['Text','Summary'],axis=1,inplace=True)

In [0]:
df.drop('Id',axis=1,inplace=True)

In [0]:
# saving back the cleaned data at particular location
df.to_csv(r'/content/drive/My Drive/Datasets/Amazon Fine Food /amazon-fine-food-reviews/Reviews_cleaned.csv',index=False)